In [1]:
import pandas as pd
import random
import numpy as np
import tensorflow as tf

/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sogol/anaconda3/lib/python3.7/site-packages/tensorflow-2.0.0b1-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint

In [2]:
train = pd.read_csv("data/freebase15k/train.txt",delimiter="\t", header=None)
test = pd.read_csv("data/freebase15k/test.txt",delimiter="\t", header=None)
validation = pd.read_csv("data/freebase15k/valid.txt",delimiter="\t", header=None)
train.columns = ['Header', 'Relation', 'Tail']
test.columns = ['Header', 'Relation', 'Tail']
validation.columns = ['Header', 'Relation', 'Tail']

In [3]:
entities = pd.concat([train.Header, train.Tail])
entities = pd.DataFrame(entities,columns=['ID'])
entities = entities.groupby('ID').ID.agg(['count']).reset_index()
# entities

In [4]:
relations = pd.DataFrame([], columns=['ID'])
relations['ID'] = train.Relation
relations = relations.groupby('ID').ID.agg(['count']).reset_index()
# relations

In [5]:
iteration = 500
dim = 100
kns = 1
alpha = 0.000001
beta = 0.0000001

nn1 = np.random.uniform(low=-0.5 / 100, high=0.5 / 100, size=(entities.shape[0], 100))
nn0 = np.random.uniform(low=-0.5 / 100, high=0.5 / 100, size=(entities.shape[0], 100))
nn2 = np.random.uniform(low=-0.5 / 100, high=0.5 / 100, size=(relations.shape[0],100))

In [6]:
totalWords = sum([freq[1] **(3/4) for freq in entities.values])     
entities['Probability'] = (entities['count']**(3/4) / totalWords)

In [7]:
def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))

In [8]:
def calculateRank(triple):
    triple=list(triple)
    _target_index = [entities.ID[ entities.ID == triple[2]].index[0]]
    _head_index = [entities.ID[entities.ID == triple[0]].index[0]]
    _relation_index = [relations.ID[relations.ID == triple[1]].index[0]]
    courapted = pd.MultiIndex.from_product([ _head_index,_relation_index, entities.iloc[:].index.tolist()], names = ["Head", "Relation", "Tail"])
    courapted =pd.DataFrame(index = courapted).reset_index()
    courapted = courapted.append(pd.DataFrame(index = pd.MultiIndex.from_product([ entities.iloc[:].index.tolist(),_relation_index,_target_index ], names = ["Head", "Relation", "Tail"])).reset_index())
    
    p = tf.math.sigmoid(tf.tensordot(tf.convert_to_tensor(nn1[courapted.Tail.values],dtype=tf.float32),tf.convert_to_tensor(np.transpose(nn0[courapted.Head.values] + nn2[courapted.Relation.values ]),dtype=tf.float32),axes=1 ))
    courapted['Score'] =tf.linalg.diag_part(p)
    courapted = courapted.sort_values(by=['Score'], ascending=False)
    courapted['Rank'] = courapted['Score'].rank(ascending=False)
    return courapted[(courapted['Head'] == _head_index[0]) & (courapted['Tail'] == _target_index[0]) & (courapted['Relation'] == _relation_index[0])].iloc[0].Rank

In [9]:
def mrr(ranks):
    inverse = []
    for rank in ranks:
        inverse.append(float(1)/rank)
    summ = sum(inverse)
    return float(1)/len(inverse) * summ

In [10]:
for x in range(0 , iteration):
    # random.shuffle(train)
    for triple in train.values:
        negative_samples = [(target, 0) for target in (np.random.choice(entities['ID'].values,size = kns, p=entities['Probability'].values))]
        classifiers = [(triple[2], 1)] + negative_samples
        random.shuffle(classifiers)
        neu1e = np.zeros(dim)
        for target, label in classifiers:
            _target_index = entities.ID[entities.ID==target ].index[0]
            _head_index = entities.ID[entities.ID == triple[0]].index[0]
            _relation_index = relations.ID[relations.ID == triple[1]].index[0]
            z = np.dot(nn1[_target_index], nn0[_head_index] + nn2[_relation_index])
            p = sigmoid(z)
            g = alpha * (label - p)
            g1 = beta * (label - p)
            neu1e += (g * nn1[_target_index] + g1 * nn2[_relation_index])
            nn1[_target_index] += g * nn0[_head_index]
            nn2[_relation_index] += g * nn0[_head_index] 
        nn0[_head_index] +=neu1e
    if x % 10 == 0 and x > 20:
        validation['Rank'] = validation.apply(calculateRank, axis=1)
        print(mrr(validation.Rank))

2021-11-20 16:40:18.431715: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-20 16:40:18.467922: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-11-20 16:40:18.468310: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560bbf8a7200 executing computations on platform Host. Devices:
2021-11-20 16:40:18.468340: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-11-20 16:40:18.506463: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3576518416 exceeds 10% of system memory.
2021-11-20 16:40:19.863409: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3576518416 exceeds 10% of system memory.
2021-11-20 16:40:20.667778: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3576518416 exceeds 10% of system memory.
2021-11-20 16